# Experiment 2D KP

### Load Model & Dataset

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import src.real_bug_dataset
from tqdm.notebook import tqdm
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.real_bug_dataset import RealBugDataset
from src.eval.loss import JointsMSELoss
from src.eval.accuracies import accuracy
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [3]:
# Sungaya Synth
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset_synth = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
                 

In [4]:
# Sungaya Real
target_dir = "../data/real_sungaya/"
out_df = pd.read_csv(os.path.join(target_dir, "realdata.csv"))
sungaya_dataset = RealBugDataset(df=out_df, root_dir=target_dir, std=sungaya_dataset_synth.std_2d   , mean=sungaya_dataset_synth.means_2d  , transform=transforms.Compose([
                                src.real_bug_dataset.ToTensor()
                                   ]))


In [5]:
sample = sungaya_dataset[3]
name = sample['file_name']
image = sample['image']
img=image.numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

name = sample['file_name']
keypoints = sample['key_points_2D']

keypoints = (sample['key_points_2D']*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

heatmap = sample['heatmap'].numpy()

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot()

ax.imshow(img.astype('uint8'))
heat =ax.imshow(cv2.resize(heatmap[0], (256,256)), alpha = 0.3,cmap='hot')
ax.scatter(keypoints[:,0], keypoints[:,1], marker='o', s=5,  color='b')

# ax.set_xlim([0,256])
# ax.set_ylim([256,0])
ax.set_aspect('equal')

plt.title(name)
fig.colorbar(heat)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
batch_size = 128
test_dataloader = DataLoader(sungaya_dataset, batch_size=batch_size)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Create Model
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 62)


criterion = JointsMSELoss(use_target_weight=True)
model = model.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

Using cuda device


In [8]:
def test_func(model,ds, thr=0.5):
    test_epoch_acc, test_epoch_loss = 0, 0
    test_epoch_acc_kps = [0]*28
    batches = 0
    # Code get the first batch of results
    with torch.no_grad():
        for data in tqdm(ds, desc="2D Keypoint Experiment"):
            input = data['image'].to(device, dtype=torch.float)
            target_heatmap = data['heatmap'].to(device, dtype=torch.float)
            heat_weight = data['heat_weight'].to(device, dtype=torch.float)

            pred_heatmap =  model(input)

            test_loss = criterion(pred_heatmap, target_heatmap, heat_weight)

            test_acc_kps, test_acc, _, _ = accuracy(pred_heatmap.detach().cpu().numpy(),
                                            target_heatmap.detach().cpu().numpy(),thr=  thr)
                                            
            test_epoch_loss += test_loss.item()
            test_epoch_acc += test_acc
            test_epoch_acc_kps += test_acc_kps[1:29]
            batches +=1
            
    test_acc = test_epoch_acc/batches
    test_acc_kps = test_epoch_acc_kps/batches
    test_loss = test_epoch_loss/batches
    return test_acc, test_loss, test_acc_kps


In [9]:
trained_model.eval()
test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader)
print(test_acc)
print(test_loss)

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

0.14074675324675323
0.001553408452309668


In [10]:
# Plots the accuracies per KP at normal threshold
 
fig, ax = plt.subplots()

ax.barh(np.array(label_names)[sungaya_dataset.reduced_kp], test_acc_kps, linewidth =5, edgecolor="white")
ax.invert_yaxis()
# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("2D PCK Accuracies for every Keypoint on Real Data)")

plt.xlabel("Accuracies")
plt.ylabel("Keypoints")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Need to do an experiment where we vary the threshold value on the test set
thresholds = [0.01 ,0.1, 0.5, 1.0, 5.0, 10.0, 20.0] # could use np.linspace if need be
# thresholds = np.linspace(0.1,5,8)
accs = []
losses = []
acc_kps = []
for thr in thresholds:
    test_acc, test_loss, test_acc_kps = test_func(trained_model, test_dataloader, thr)
    accs.append(test_acc)
    losses.append(test_loss)
    acc_kps.append(test_acc_kps)

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

2D Keypoint Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# plots PCK for different thresholds
fig, ax = plt.subplots()

ax.bar(list(map(str,thresholds)), accs, width=1, edgecolor="white", linewidth=5)

# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("2D PCK Accuracies at Varying Thresholds on Real Data")
plt.xlabel("Thresholds")
plt.ylabel("Accuracies")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader:
        image = data['image']
        target = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(device, dtype=torch.float)
        heatmap_var = target.to(device, dtype=torch.float)

        predi = trained_model(input_var)


In [14]:
from src.eval.accuracies import get_max_preds
# Display a sample
sample = 2
limb = 22

name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = predi.cpu().numpy()

acc = target.cpu().numpy()
acc_kp = ((kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d).astype(int)

vis = visib[sample].cpu().numpy()
print("Limb Visible", vis[limb] , acc_kp[limb])
plt.figure(figsize=(8,6))
plt.imshow(img)

plt.scatter(acc_kp[limb,0], acc_kp[limb,1], marker='o', s=10,  color='b')

plt.imshow(cv2.resize(pred[sample,limb], (256,256)), alpha = .4,cmap='hot')

plt.colorbar()
plt.title(name)
plt.show()

Limb Visible 1 [ 97 105]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …